---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# train_data = pd.read_csv('train.csv', encoding='latin1')
# test_data = pd.read_csv('test.csv', encoding='latin1')
# address = pd.read_csv('addresses.csv')
# latlons = pd.read_csv('latlons.csv')
# locs = pd.merge(address[['ticket_id', 'address']], latlons[['lat', 'lon', 'address']], how='inner', on='address')
# columns = ['ticket_id', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'judgment_amount'] # select variables for training
# train_data = train_data[columns+['compliance']]
# train_data = train_data[train_data['compliance'].notnull()]
# train_data = pd.merge(train_data, locs, how='left', on='ticket_id')
# test_data = pd.merge(test_data, locs, how='left', on='ticket_id')
# train_data = train_data.dropna()
# test_data = test_data.dropna()
# train_columns = ['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'judgment_amount', 'lat', 'lon']
# X_train = train_data[train_columns]
# target_column = ['compliance']
# y_train = train_data[target_column]
def blight_model():
    """
    4 datasets: train.csv, test.csv, addresses.csv, latlons.csv
    """
    # Your code here
    
    # load datasets
    train_data = pd.read_csv('train.csv', encoding='latin1')
    test_data = pd.read_csv('test.csv', encoding='latin1')
    address = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    locs = pd.merge(address[['ticket_id', 'address']], latlons[['lat', 'lon', 'address']], how='inner', on='address')
    
    # preprocess
    train_data = train_data[train_data['compliance'].notnull()]
    columns = ['ticket_id', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'judgment_amount'] # select variables for training
    train_data = train_data[columns+['compliance']]
    test_data = test_data[columns]
    train_data = pd.merge(train_data, locs, how='left', on='ticket_id')
    test_data = pd.merge(test_data, locs, how='left', on='ticket_id')
    train_data = train_data.dropna()
    test_data = test_data.dropna()
    train_data = train_data.drop_duplicates('ticket_id', keep = 'last') # in case of duplicates
    test_data = test_data.drop_duplicates('ticket_id', keep = 'last') # in case of duplicates
    
    # train
    train_columns = ['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'judgment_amount', 'lat', 'lon']
    target_column = ['compliance']
    X_train = train_data[train_columns].astype('float32')
    y_train = train_data[target_column].astype('float32')
    clf = RandomForestClassifier(n_estimators=100, n_jobs=2, random_state=0) 
    clf.fit(X_train, y_train)
    accuracy_score = clf.score(X_train, y_train) # 0.988078409787463
    
    # test
    X_test = test_data[train_columns].astype('float32')
    probs = clf.predict_proba(X_test)
    result = pd.Series(data = probs[:, 1], index = test_data['ticket_id'])
    return result # Your answer here

In [2]:
blight_model()

C:\Users\hdeng8\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3220: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
C:\Users\hdeng8\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:52: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


ticket_id
284932    0.020000
285362    0.000000
285361    0.010000
285338    0.070000
285346    0.000000
285345    0.060000
285347    0.020000
285342    0.380000
285530    0.018690
284989    0.030000
285344    0.000000
285343    0.000000
285340    0.190000
285341    0.285333
285349    0.040000
285348    0.010000
284991    0.010000
285532    0.160000
285406    0.010000
285001    0.000000
285006    0.000000
285405    0.010000
285337    0.020000
285496    0.000000
285497    0.060000
285378    0.000000
285589    0.000000
285585    0.000000
285501    0.116667
285581    0.220000
            ...   
376367    0.000000
376366    0.040000
376362    0.010000
376363    0.030000
376365    0.000000
376364    0.040000
376228    0.820000
376265    0.030000
376286    0.595000
376320    0.070500
376314    0.000000
376327    0.780000
376385    0.360000
376435    0.430000
376370    0.450000
376434    0.000000
376459    0.010000
376478    0.000000
376473    0.018000
376484    0.030000
376482    0.010000
37